# Round Trip Routes

## Goals
* To get a suggested round trip route for a certain distance or duration. Useful to plan for exercise, such as taking a 30min walk around the office.
* To learn about OpenStreetMap and related libraries for routing.

## Current Algorithm
* Search for an isochrone for half the desired distance or duration.
* Pick a random one and navigate there and back. Use alternate routes (only when available) to avoid retracing your steps.
* Route factor in walking speed and other parameters that already exist in Valhalla API.

## Future considerations
* Avoid traffic lights (need to query OpenStreetMap for traffic lights in the area then add them to "avoid_locations" parameter)
* Favor shaded area

# Dependencies

* Uses **[geopy](https://geopy.readthedocs.io/en/stable/)** to search **[Nominatim](https://nominatim.openstreetmap.org/ui/search.html)**
  * Nominatim is a search engine for OpenStreetMap data.
* Plot with **[ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)**
  * Ipyleaflet is a Jupyter widget for Leaflet.js
* Uses **[routingpy](https://routingpy.readthedocs.io/en/latest/#)** against a locally running **[Valhalla](https://valhalla.github.io/valhalla/)**
  * Valhalla is an open source routing engine and accompanying libraries for use with OpenStreetMap data
  * Run a server with a norcal dataset for OpenStreetMap
    ```bash
    docker run --rm --name valhalla_gis-ops -p 8002:8002 -v $PWD/custom_files:/custom_files -e tile_urls=https://download.geofabrik.de/north-america/us/california/norcal-latest.osm.pbf ghcr.io/gis-ops/docker-valhalla/valhalla:latest
    ```

In [48]:
#!pip install -r requirements.txt

In [49]:
from geopy import Location, Nominatim
from ipyleaflet import AwesomeIcon, Map, Marker, Polygon, Polyline, Popup
from ipywidgets import HTML
from routingpy import Valhalla
from pprint import pprint
from random import choice

# Search for a start location

In [50]:
# Don't abuse

# geolocator = Nominatim(user_agent="chris_test")
# start = geolocator.geocode("Toyota Research Institute")

In [51]:
# Saved result for TRI HQ
start = Location(
    "Toyota Research Institute, 4440, West El Camino Real, Los Altos, Santa Clara County, California, 94022, United States", 
    (37.40253645, -122.1165510679842, 0.0),
    {
        'place_id': 311529403,
         'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
         'osm_type': 'way',
         'osm_id': 256656808,
         'lat': '37.40253645',
         'lon': '-122.1165510679842',
         'class': 'building',
         'type': 'commercial',
         'place_rank': 30,
         'importance': 9.99999999995449e-06,
         'addresstype': 'building',
         'name': 'Toyota Research Institute',
         'display_name': 'Toyota Research Institute, 4440, West El Camino Real, Los Altos, Santa Clara County, California, 94022, United States',
         'boundingbox': ['37.4021493', '37.4029503', '-122.1171370', '-122.1159999']
    }
)

In [52]:
start_lat_long = (start.latitude, start.longitude)
start_long_lat = (start.longitude, start.latitude)

In [53]:
startpoint_icon = AwesomeIcon(
    name='home',
    marker_color='green',
    icon_color='black',
    spin=False
)

center_map = Map(center=start_lat_long, zoom=15)

startpoint_marker = Marker(location=start_lat_long, draggable=False, icon=startpoint_icon)
center_map.add_control(startpoint_marker)
center_map

Map(center=[37.40253645, -122.1165510679842], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

# Get Isochrones for Time or Distance

In [54]:
MPH_TO_KPH = 1.60934
METERS_PER_MILE = 1609.34
SEC_PER_MIN = 60

In [55]:
desired_round_trip_time_min = 25
desired_round_trip_distance_miles = 1
desired_speed = 2.5 * MPH_TO_KPH

# https://valhalla.github.io/valhalla/api/turn-by-turn/api-reference/#pedestrian-costing-options
walking_options = {
    "walking_speed": desired_speed,

    # higher value is higher cost (less favored)
    "walkway_factor": 1,
    "sidewalk_factor": 1,
    "alley_factor": 2,
    "driveway_factor": 5,
    "step_penalty": 1,

    # 0 to 1, where 1 is more favored
    "use_living_streets": 0.6,
    "use_tracks": 0.5,
    "use_hills": 0.5,
    "use_lit": 0
}

In [56]:
isochrone_half = desired_round_trip_time_min * SEC_PER_MIN / 2
equidistant_half = desired_round_trip_distance_miles * METERS_PER_MILE / 2

halfway_isochrones = Valhalla('http://localhost:8002').isochrones(
    locations=start_long_lat, 
    profile="pedestrian",
    units="mi",
    options=walking_options,
    intervals=[isochrone_half],
    # intervals=[equidistant_half],
    interval_type="time"
)


halfway_points = [(lat, long) for (long, lat) in halfway_isochrones[0].geometry]
halfway_area = Polygon(
    locations=halfway_points,
    color="blue",
    fill_color="blue"
)

halfway_map = Map(center=start_lat_long, zoom=15)
halfway_map.add(halfway_area);
halfway_map.add(startpoint_marker)
halfway_map

Map(center=[37.40253645, -122.1165510679842], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

# Pick a waypoint

In [57]:
waypoint_icon = AwesomeIcon(
    name='flag',
    marker_color='red',
    icon_color='black',
    spin=False
)
waypoint_marker = Marker(location=start_lat_long, draggable=True, icon=waypoint_icon)
routing_map = Map(center=start_lat_long, zoom=15)
routing_map.add_control(startpoint_marker)
routing_map.add_control(waypoint_marker)
display(routing_map)

all_instructions = []
total_distance = 0 
total_duration = 0
all_geometry = []
line = None
popup = None

Map(center=[37.40253645, -122.1165510679842], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

# Get a Route

In [58]:
# random
routing_map.remove(waypoint_marker)
waypoint_lat_long = choice(halfway_points)
waypoint_marker = Marker(location=waypoint_lat_long, draggable=True, icon=waypoint_icon)
routing_map.add(waypoint_marker)

# # from choice
# waypoint_lat_long = waypoint_marker.location

if line:
    routing_map.remove(line)
if popup:
    routing_map.remove(popup)

waypoint_long_lat = waypoint_lat_long[1], waypoint_lat_long[0]

#six degrees of precision in valhalla
inv = 1.0 / 1e6;

#decode an encoded string
def decode_shape(encoded):
  decoded = []
  previous = [0,0]
  i = 0
  #for each byte
  while i < len(encoded):
    #for each coord (lat, lon)
    ll = [0,0]
    for j in [0, 1]:
      shift = 0
      byte = 0x20
      #keep decoding bytes until you have this coord
      while byte >= 0x20:
        byte = ord(encoded[i]) - 63
        i += 1
        ll[j] |= (byte & 0x1f) << shift
        shift += 5
      #get the final value adding the previous offset and remember it for the next
      ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
      previous[j] = ll[j]
    #scale by the precision and chop off long coords also flip the positions so
    #its the far more standard lon,lat instead of lat,lon
    decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
  #hand back the list of coordinates
  return decoded

def get_directions(long_lat_1, long_lat_2):
    directions = Valhalla('http://localhost:8002').directions(
        locations=[long_lat_1, long_lat_2], 
        profile="pedestrian",
        units="mi",
        options=walking_options,
        instructions=True,
        alternates=2,
    )
    if "alternates" in directions.raw:
        raw_trip = directions.raw["alternates"][0]["trip"]
    else:
        raw_trip = directions.raw["trip"]
        
    instructions = [f"{m['instruction']} Continue for {int(m['length'] * 5280)} feet" for m in raw_trip["legs"][0]["maneuvers"]]
    distance = raw_trip["summary"]["length"]
    duration = raw_trip["summary"]["time"] / 60
    geometry = decode_shape(raw_trip["legs"][0]["shape"])
    return instructions, distance, duration, geometry

all_instructions = []
total_distance = 0 
total_duration = 0
all_geometry = []
instructions, distance, duration, geometry = get_directions(start_long_lat, waypoint_long_lat)
all_instructions.extend(instructions)
total_distance += distance
total_duration += duration
all_geometry.extend(geometry)
instructions, distance, duration, geometry = get_directions(waypoint_long_lat, start_long_lat)
all_instructions.extend(instructions)
total_distance += distance
total_duration += duration
all_geometry.extend(geometry)

locations = [(lat, long) for (long, lat) in all_geometry]
line = Polyline(
    locations=locations,
    color="green" ,
    fill=False
)

routing_map.add(line)

print("Duration (minutes):", total_duration)
print("Distance (miles):", total_distance)
pprint(all_instructions)

message = HTML()
message.value = f"{int(total_duration)} minutes; {total_distance} miles"

# Popup with a given location on the map:
popup = Popup(
    location=waypoint_lat_long,
    child=message,
    close_button=True,
    auto_close=False,
    close_on_escape_key=True
)
routing_map.add(popup)


Duration (minutes): 30.1965
Distance (miles): 1.248
['Walk southeast. Continue for 174 feet',
 'Make a sharp left. Continue for 42 feet',
 'Turn right onto the walkway. Continue for 749 feet',
 'Turn left onto the crosswalk. Continue for 84 feet',
 'Turn right onto North San Antonio Road. Continue for 1642 feet',
 'Turn right onto West Portola Avenue. Continue for 205 feet',
 'Turn left. Continue for 15 feet',
 'Turn left onto the walkway. Continue for 401 feet',
 'Your destination is on the right. Continue for 0 feet',
 'Walk north on the walkway. Continue for 401 feet',
 'Turn right. Continue for 15 feet',
 'Turn right onto West Portola Avenue. Continue for 205 feet',
 'Turn left onto North San Antonio Road. Continue for 1404 feet',
 'Turn left. Continue for 21 feet',
 'Turn right onto the walkway. Continue for 227 feet',
 'Keep left to take the walkway. Continue for 733 feet',
 'Turn left. Continue for 42 feet',
 'Make a sharp right. Continue for 174 feet',
 'Your destination is on 

Map(bottom=3253522.0, center=[37.40253645, -122.1165510679842], controls=(ZoomControl(options=['position', 'zo…

# Optimized Route through a number of waypoints

In [114]:
NUM_WAYPOINTS = 3
waypoint_icon = AwesomeIcon(
    name='flag',
    marker_color='blue',
    icon_color='black',
    spin=False
)
waypoint_markers = [Marker(location=start_lat_long, draggable=True, icon=waypoint_icon) for _ in range(NUM_WAYPOINTS)]
opt_routing_map = Map(center=start_lat_long, zoom=15)
opt_routing_map.add_control(startpoint_marker)
for waypoint_marker in waypoint_markers:
    opt_routing_map.add_control(waypoint_marker)
display(opt_routing_map)

Map(center=[37.40253645, -122.1165510679842], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

In [117]:
import requests
import json

optimized_line = None

v = Valhalla('http://localhost:8002')

In [123]:
locations = [{"lat": start.latitude, "lon": start.longitude}]+[{"lat": w.location[0], "lon": w.location[1]} for w in waypoint_markers]+[{"lat": start.latitude, "lon": start.longitude}]

json_params = {"locations":locations,"costing":"pedestrian","directions_options":{"units":"miles"}}
response = requests.get(f'http://localhost:8002/optimized_route?json={json.dumps(json_params)}')

lon_lats = []
for leg in response.json()['trip']['legs']:
    lon_lats.extend(decode_shape(leg['shape']))
lat_lons = [(lat, lon) for lon, lat in lon_lats]

if optimized_line:
    opt_routing_map.remove(optimized_line)
optimized_line = Polyline(
    locations=lat_lons,
    color="green" ,
    fill=False
)
opt_routing_map.add(optimized_line)

Map(bottom=6506744.0, center=[37.40425557926452, -122.11575150489809], controls=(ZoomControl(options=['positio…